In [274]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

df=pd.DataFrame(df[0])

**Here I need to do some data transfomation to get the dataset that I can used for the cluster analysis**

In [275]:
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [276]:
df=df[df.Borough!='Not assigned']
df.reset_index(inplace=True)

In [277]:
for i in range(df.shape[0]):
    if df.loc[i,'Neighbourhood']=='Not assigned':
        df.loc[i,'Neighbourhood']=df.loc[i,'Borough']

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [278]:
df = df.groupby('Borough').agg({'Postcode':'first',
                             'Neighbourhood': ', '.join 
                             }).reset_index()
df

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For..."
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis..."
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, The..."
3,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig..."
4,Etobicoke,M9A,"Islington Avenue, Cloverdale, Islington, Marti..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,..."
7,Queen's Park,M7A,Queen's Park
8,Scarborough,M1B,"Rouge, Malvern, Highland Creek, Rouge Hill, Po..."
9,West Toronto,M6H,"Dovercourt Village, Dufferin, Little Portugal,..."


In [279]:
df.shape


(11, 3)

**Above is the first dataset that I got for the different Borough with postal code.  
Below I need to get the lat and long data for each postal code to do the mapping and merging.**

In [280]:
df1=pd.read_csv('https://cocl.us/Geospatial_data')
df1

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [281]:
df1=df1.rename(columns={'Postal Code':'Postcode'})
df2=pd.merge(df,df1,on='Postcode')

In [282]:
df2

,Borough,Postcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For...",43.728020,-79.388790
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis...",43.654260,-79.360636
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, The...",43.676357,-79.293031
3,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig...",43.706397,-79.309937
4,Etobicoke,M9A,"Islington Avenue, Cloverdale, Islington, Marti...",43.667856,-79.532242
5,Mississauga,M7R,Canada Post Gateway Processing Centre,43.636966,-79.615819
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,...",43.753259,-79.329656
7,Queen's Park,M7A,Queen's Park,43.662301,-79.389494
8,Scarborough,M1B,"Rouge, Malvern, Highland Creek, Rouge Hill, Po...",43.806686,-79.194353
9,West Toronto,M6H,"Dovercourt Village, Dufferin, Little Portugal,...",43.669005,-79.442259


**Below I would add some mapping information for the segment and clustering to show how  
those areas from 'Borough' are different from each other.**

In [283]:
from geopy.geocoders import Nominatim
address = 'Toronto,CA'

geolocator = Nominatim(user_agent="TR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

**After setting the location, lat, and long, lets see the mapping and the segment results. **

In [284]:
import folium
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

**Now it's time to insert the foursquare id and secret to do the cluster analysis**

In [285]:
CLIENT_ID = 'P5BWDSBVKYNFXKXUGPUUDF0NGO11CK5UJIQOF00M0W5JHRBC' # your Foursquare ID
CLIENT_SECRET = '1FW5RC1OGWWZYLUM0PYVWMHCNAIQX4OCFF2G15JDFAADOW03' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P5BWDSBVKYNFXKXUGPUUDF0NGO11CK5UJIQOF00M0W5JHRBC
CLIENT_SECRET:1FW5RC1OGWWZYLUM0PYVWMHCNAIQX4OCFF2G15JDFAADOW03


In [286]:
df2 = df2[(df2['Borough'] == 'Central Toronto') | (df2['Borough']=='Downtown Toronto') | (df2['Borough']=='North York')\
         | (df2['Borough']=='East York')| (df2['Borough']=='York')| (df2['Borough']=='Beaches-East York')].reset_index(drop=True)
df2_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
df2_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

df2_name = df2.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(df2_name, 
                                                               df2_latitude, 
                                                               df2_longitude))

Latitude and longitude values of Lawrence Park, Roselawn, Davisville North, Forest Hill North, Forest Hill West, North Toronto West, The Annex, North Midtown, Yorkville, Davisville, Moore Park, Summerhill East, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West are 43.7280205, -79.3887901.


In [287]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df2_latitude, 
    df2_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=P5BWDSBVKYNFXKXUGPUUDF0NGO11CK5UJIQOF00M0W5JHRBC&client_secret=1FW5RC1OGWWZYLUM0PYVWMHCNAIQX4OCFF2G15JDFAADOW03&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [288]:
results = requests.get(url).json()

In [289]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Here I want to get the categories to see the patterns of the name**

In [290]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [291]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [292]:
tor_venues = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )




Lawrence Park, Roselawn, Davisville North, Forest Hill North, Forest Hill West, North Toronto West, The Annex, North Midtown, Yorkville, Davisville, Moore Park, Summerhill East, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Harbourfront, Regent Park, Ryerson, Garden District, St. James Town, Berczy Park, Central Bay Street, Christie, Adelaide, King, Richmond, Harbourfront East, Toronto Islands, Union Station, Design Exchange, Toronto Dominion Centre, Commerce Court, Victoria Hotel, Harbord, University of Toronto, Chinatown, Grange Park, Kensington Market, CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Rosedale, Stn A PO Boxes 25 The Esplanade, Cabbagetown, St. James Town, First Canadian Place, Underground city, Church and Wellesley
Woodbine Gardens, Parkview Hill, Woodbine Heights, Leaside, Thorncliffe Park, East Toronto
Parkwoods, Victoria Village, Lawrence Heights, Lawrence Manor, Don Mills North, Glenca

In [293]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Harbourfront, Regent Park, Ryerson, Garden District, St. James Town, Berczy Park, Central Bay Street, Christie, Adelaide, King, Richmond, Harbourfront East, Toronto Islands, Union Station, Design Exchange, Toronto Dominion Centre, Commerce Court, Victoria Hotel, Harbord, University of Toronto, Chinatown, Grange Park, Kensington Market, CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Rosedale, Stn A PO Boxes 25 The Esplanade, Cabbagetown, St. James Town, First Canadian Place, Underground city, Church and Wellesley",51,51,51,51,51,51
"Humewood-Cedarvale, Caledonia-Fairbanks, Del Ray, Keelesdale, Mount Dennis, Silverthorn, The Junction North, Runnymede, Weston",4,4,4,4,4,4
"Lawrence Park, Roselawn, Davisville North, Forest Hill North, Forest Hill West, North Toronto West, The Annex, North Midtown, Yorkville, Davisville, Moore Park, Summerhill East, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",3,3,3,3,3,3
"Parkwoods, Victoria Village, Lawrence Heights, Lawrence Manor, Don Mills North, Glencairn, Flemingdon Park, Don Mills South, Hillcrest Village, Bathurst Manor, Downsview North, Wilson Heights, Fairview, Henry Farm, Oriole, Northwood Park, York University, Bayview Village, CFB Toronto, Downsview East, Silver Hills, York Mills, Downsview West, Downsview, North Park, Upwood Park, Humber Summit, Newtonbrook, Willowdale, Downsview Central, Bedford Park, Lawrence Manor East, Emery, Humberlea, Willowdale South, Downsview Northwest, York Mills West, Willowdale West",3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill, Woodbine Heights, Leaside, Thorncliffe Park, East Toronto",11,11,11,11,11,11


**Now we can look at the different categories for different neighborhood, if there is something that already exist in this area, it is not that recommended to open a business around that area since there might be more competitions.**

In [294]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
new_col=list(tor_onehot.columns)

new_col.remove('Neighborhood')
fixed_columns = ['Neighborhood'] + new_col
tor_onehot = tor_onehot[fixed_columns]
tor_onehot


,Neighborhood,Antique Shop,Art Gallery,Athletics & Sports,Bakery,Bank,Beer Store,Breakfast Spot,Brewery,Bus Line,...,Pharmacy,Pizza Place,Pub,Restaurant,Shoe Store,Spa,Swim School,Theater,Trail,Yoga Studio
0,"Lawrence Park, Roselawn, Davisville North, For...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Lawrence Park, Roselawn, Davisville North, For...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,"Lawrence Park, Roselawn, Davisville North, For...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park, Ryerson, Garden Dis...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park, Ryerson, Garden Dis...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Parkwoods, Victoria Village, Lawrence Heights,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Antique Shop,Art Gallery,Athletics & Sports,Bakery,Bank,Beer Store,Breakfast Spot,Brewery,Bus Line,...,Pharmacy,Pizza Place,Pub,Restaurant,Shoe Store,Spa,Swim School,Theater,Trail,Yoga Studio
0,"Harbourfront, Regent Park, Ryerson, Garden Dis...",0.019608,0.019608,0.000000,0.058824,0.019608,0.019608,0.039216,0.019608,0.000000,...,0.000000,0.000000,0.058824,0.039216,0.019608,0.019608,0.000000,0.039216,0.00,0.019608
1,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000
2,"Lawrence Park, Roselawn, Davisville North, For...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000
3,"Parkwoods, Victoria Village, Lawrence Heights,...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"Woodbine Gardens, Parkview Hill, Woodbine Heig...",0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,...,0.090909,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [296]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Harbourfront, Regent Park, Ryerson, Garden District, St. James Town, Berczy Park, Central Bay Street, Christie, Adelaide, King, Richmond, Harbourfront East, Toronto Islands, Union Station, Design Exchange, Toronto Dominion Centre, Commerce Court, Victoria Hotel, Harbord, University of Toronto, Chinatown, Grange Park, Kensington Market, CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Rosedale, Stn A PO Boxes 25 The Esplanade, Cabbagetown, St. James Town, First Canadian Place, Underground city, Church and Wellesley----
         venue  freq
0  Coffee Shop  0.18
1         Café  0.06
2       Bakery  0.06
3         Park  0.06
4          Pub  0.06


----Humewood-Cedarvale, Caledonia-Fairbanks, Del Ray, Keelesdale, Mount Dennis, Silverthorn, The Junction North, Runnymede, Weston----
           venue  freq
0          Trail  0.25
1   Hockey Arena  0.25
2          Field  0.25
3  Moving Target  0.25
4   Antique Shop  0.00


----Lawren

**Now we can get to know the top 3 venues around different neighbors,  
this would help a lot in making business decision as well.**

In [297]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [298]:
import numpy as np
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Harbourfront, Regent Park, Ryerson, Garden Dis...",Coffee Shop,Café,Pub
1,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",Trail,Field,Hockey Arena
2,"Lawrence Park, Roselawn, Davisville North, For...",Swim School,Bus Line,Park
3,"Parkwoods, Victoria Village, Lawrence Heights,...",Food & Drink Shop,Fast Food Restaurant,Park
4,"Woodbine Gardens, Parkview Hill, Woodbine Heig...",Fast Food Restaurant,Pizza Place,Gym / Fitness Center


**Merge the data with the original one with lat and long in order to do a mapping below.**

In [299]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [300]:
# set number of clusters
kclusters = 4

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 3, 1, 2], dtype=int32)

In [301]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

tor_merged = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
#tor_merged.Cluster_Labels=tor_merged.Cluster_Labels.astype(int)
tor_merged.head() # check the last columns!

,Borough,Postcode,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For...",43.728020,-79.388790,3,Swim School,Bus Line,Park
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis...",43.654260,-79.360636,2,Coffee Shop,Café,Pub
2,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig...",43.706397,-79.309937,2,Fast Food Restaurant,Pizza Place,Gym / Fitness Center
3,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,...",43.753259,-79.329656,1,Food & Drink Shop,Fast Food Restaurant,Park
4,York,M6C,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",43.693781,-79.428191,0,Trail,Field,Hockey Arena


**Now we can figure out which neighbors belong to the same cluster based on the patterns shown above.   
This could help in making business decision more accurate since this one could show more patterns between different clusters.  
If the neighbors belong to same cluster, the characteristic might be similar, we can find the central part to open a business, based on the venues data shown above.**

In [302]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters